In [ ]:
import pandas as pd
from glob import glob
import cv2
from tqdm import tqdm

In [ ]:
test_jpg_dir = sorted(glob('./data/test/*/*.jpg'))
test_csv_dir = sorted(glob('./data/test/*/*.csv'))

In [ ]:
def copy_img_csv(_dir, folder_name): # dataset: 'train' or 'test'
for i, jpg in enumerate(tqdm(_dir)):
    img = cv2.imread(jpg)
    img_name = jpg.split('\\')[-1]
    cv2.imwrite(f'./data/{folder_name}/images/{img_name}', img)
    csv = pd.read_csv(csv_dir[i])
    csv.to_csv(f'./data/{folder_name}/csv/{img_name.replace(".jpg",".csv")}')

In [ ]:
train_csv_dir = sorted(glob('./data/train/train/*/*.csv'))

# csv분석

In [27]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')

import warnings
warnings.filterwarnings('ignore')

# import torch
# from torch import nn
# from torchvision import models
# from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import platform
import sys
import os
import sklearn 
import cv2
import json
from glob import glob

print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")
print(f"- sklearn: {sklearn.__version__}")
# print(f"- pytorch: {torch.__version__}")
print(f"- opencv: {cv2.__version__}")

- os: Windows-10-10.0.22000-SP0
- python: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
- pandas: 1.3.5
- numpy: 1.21.5
- sklearn: 1.0.2
- opencv: 4.5.5


In [28]:
TRAIN_PATH = 'C:/Users/jeong/Projects/dacon/병해진단/data/train/'

In [29]:
train_csv = sorted(glob(TRAIN_PATH +'*/*.csv'))
train_jpg = sorted(glob(TRAIN_PATH +'*/*.jpg'))
train_json = sorted(glob(TRAIN_PATH +'*/*.json'))

# test_csv = sorted(glob(TRAIN_PATH +'*/*.csv'))
# test_jpg = sorted(glob(TRAIN_PATH +'*/*.jpg'))
# test_json = sorted(glob(TRAIN_PATH +'*/*.json'))

In [30]:
crops = []
areas = []
diseases = []
risks = []
labels = []
grows = [] 
parts=[]
objects = []
img_names=[]
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        
        crop = sample['annotations']['crop']
        area = sample['annotations']['area']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"        
        grow = sample['annotations']['grow']
        img_name = sample['description']['image']
        grows.append(grow)
        crops.append(crop)
        areas.append(area)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)
        img_names.append(img_name)
        
        r = sample['annotations']['bbox'][0]
        temp = [int(r['x']), int(r['y']), int(r['w']), int(r['h'])]
        objects.append(temp)    
        
        p = []
        
        for part in sample['annotations']['part']:
            p.append({
                'bbox': [
                    int(part['x']), int(part['y']), int(part['w']), int(part['h'])
                ],
                'id': part['id']
            })
        parts.append(p)

In [31]:

df = pd.DataFrame({'image':img_names, 
                   'crops':crops,'areas':areas,'diseases':diseases,
                   'risks':risks, 'bbox':objects,'grows':grows, 'labels':labels
                  })

df['x'] = df.bbox.apply(lambda x: x[0])
df['y'] = df.bbox.apply(lambda x: x[1])
df['w'] = df.bbox.apply(lambda x: x[2])
df['h'] = df.bbox.apply(lambda x: x[3])
df['xc'] = df.x + df.w.apply(lambda x: x//2)
df['yc'] = df.y + df.w.apply(lambda x: x//2)

df= df[['image','crops','areas','diseases','risks', 'grows','xc','yc','w','h']]
df

,image,crops,areas,diseases,risks,grows,xc,yc,w,h
0,10027.jpg,3,3,b7,1,13,171,244,165,237
1,10037.jpg,3,5,00,0,11,190,26,48,315
2,10043.jpg,3,1,00,0,13,191,252,272,271
3,10045.jpg,3,3,00,0,11,214,274,172,263
4,10063.jpg,3,1,00,0,13,184,288,285,283
...,...,...,...,...,...,...,...,...,...,...
5762,67640.jpg,3,5,00,0,11,131,80,160,370
5763,67644.jpg,1,3,00,0,11,401,294,560,488
5764,67647.jpg,2,3,a5,2,12,374,377,322,174
5765,67649.jpg,1,3,00,0,12,261,145,220,239


In [ ]:
# 1. dropduplicates
# 2. sort_values
# 3. 

In [70]:
from datetime import datetime, timedelta

csv_data = []
for csv_file in tqdm(train_csv):
    data = pd.read_csv(f'{csv_file}')
    data = data[['측정시각', '내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저','내부 습도 1 평균', '내부 습도 1 최고',
       '내부 습도 1 최저','내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']]
    data = data.drop_duplicates(subset=['측정시각'])
    data = data.sort_values(by='측정시각')
    data['측정시각'] = data['측정시각'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
    csv_data.append(data)

100%|██████████████████████████████████████████████████████████████████████████████| 5767/5767 [02:45<00:00, 34.95it/s]


In [22]:
print(np.any([1,1,1]))
print(np.any([1,0,0,1]))

True
True


In [85]:
train_csv[0]

'C:/Users/jeong/Projects/dacon/병해진단/data/train\\10027\\10027.csv'

In [68]:
csv_data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   측정시각        588 non-null    object 
 1   내부 온도 1 평균  588 non-null    float64
 2   내부 온도 1 최고  588 non-null    float64
 3   내부 온도 1 최저  588 non-null    float64
 4   내부 습도 1 평균  588 non-null    float64
 5   내부 습도 1 최고  588 non-null    float64
 6   내부 습도 1 최저  588 non-null    float64
 7   내부 이슬점 평균   588 non-null    float64
 8   내부 이슬점 최고   588 non-null    float64
 9   내부 이슬점 최저   588 non-null    float64
dtypes: float64(9), object(1)
memory usage: 46.1+ KB


In [69]:
csv_data[0]

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저
0,2021-11-26 09:30:00,9.0,9.6,8.5,81.5,82.0,80.8,6.1,6.9,5.9
1,2021-11-26 09:30:00,8.7,9.3,8.1,85.4,85.7,84.9,6.3,6.8,5.7
2,2021-11-26 09:20:00,7.9,8.4,7.4,81.9,82.7,81.6,5.0,5.1,4.3
3,2021-11-26 09:20:00,7.4,7.9,6.9,86.9,87.4,85.9,5.4,6.0,5.0
4,2021-11-26 09:10:00,6.9,7.2,6.6,83.3,84.0,82.7,4.4,4.5,4.3
...,...,...,...,...,...,...,...,...,...,...
583,2021-11-24 09:00:00,8.7,8.9,8.4,89.7,90.1,89.2,7.3,7.5,6.5
584,2021-11-24 08:50:00,8.8,9.0,8.7,85.3,85.5,85.0,6.6,6.8,6.6
585,2021-11-24 08:50:00,8.2,8.4,8.0,89.9,90.1,89.6,6.5,6.5,6.5
586,2021-11-24 08:40:00,8.5,8.6,8.4,85.5,85.8,85.3,6.4,6.8,5.8


In [75]:
temp = csv_data[0].copy()
temp['ew'] = temp['내부 온도 1 평균']

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저
586,2021-11-24 08:40:00,8.5,8.6,8.4,85.5,85.8,85.3,6.4,6.8,5.8
584,2021-11-24 08:50:00,8.8,9.0,8.7,85.3,85.5,85.0,6.6,6.8,6.6
582,2021-11-24 09:00:00,9.3,9.5,9.1,85.3,85.8,84.6,6.8,7.6,6.6
580,2021-11-24 09:10:00,9.7,9.9,9.5,83.8,84.5,83.0,7.5,7.6,7.3
578,2021-11-24 09:20:00,10.0,10.1,9.9,82.4,83.0,82.0,7.2,7.3,7.1
...,...,...,...,...,...,...,...,...,...,...
8,2021-11-26 08:50:00,5.3,5.5,5.1,88.7,88.8,88.5,3.4,4.3,3.3
6,2021-11-26 09:00:00,6.4,6.6,6.2,83.8,84.2,83.6,3.7,4.5,3.5
4,2021-11-26 09:10:00,6.9,7.2,6.6,83.3,84.0,82.7,4.4,4.5,4.3
2,2021-11-26 09:20:00,7.9,8.4,7.4,81.9,82.7,81.6,5.0,5.1,4.3


In [ ]:
# 포화 수증기압
et = (6.1078*10**(7.5*temperature/(temperature+237.3)))

#포화 수증기량
at = 217* et/(temperature+273.15)

#상대습도
상대습도 = et/at *100

# 현재 대기 중의 수증기량을 현재 온도의 포화 수증기량으로 나눈 비율(%)로 나타낸다
# 현재 수증기/현재온도에서의 수증기량 *100

In [76]:
temp = csv_data[0].copy()
temp['et'] = temp['내부 온도 1 평균'].apply(lambda x: (6.1078*10**(7.5*x)/(x+237.3)))
temp.et

586    1.397343e+62
584    2.481837e+64
582    1.392810e+68
580    1.390554e+71
578    2.469794e+73
           ...     
8      1.415774e+38
6      2.506278e+46
4      1.406498e+50
2      4.429598e+57
0      7.841884e+65
Name: et, Length: 294, dtype: float64

In [80]:
temp['at'] =  temp.et.apply(lambda x: 217*x)/ temp['내부 온도 1 평균'].apply(lambda x: x+273.15)
temp.at

In [84]:
temp['상대습도'] = temp['et']/temp['at']
temp['상대습도']

586    1.297926
584    1.299309
582    1.301613
580    1.303456
578    1.304839
         ...   
8      1.283180
6      1.288249
4      1.290553
2      1.295161
0      1.300230
Name: 상대습도, Length: 294, dtype: float64

In [83]:
temp['at']

586    1.076596e+62
584    1.910121e+64
582    1.070064e+68
580    1.066821e+71
578    1.892796e+73
           ...     
8      1.103333e+38
6      1.945492e+46
4      1.089841e+50
2      3.420113e+57
0      6.031149e+65
Name: at, Length: 294, dtype: float64

In [73]:
np.exp(0)

1.0

In [ ]:
217* 

In [ ]:
crop_dict = {1:'딸기', 
             2:'토마토',
             3:'파프리카',
             4:'오이', 
             5:'고추',
             6: '시설포도'}

area_dict = {1:'열매',
             2:'꽃',
             3:'잎',
             4:'가지',
             5:'줄기',
             6:'뿌리',
             7:'해충'}

task_dict = {0:'정상',
             1:'병해',
             2:'생리장애',
             3:'보호제처리반응'}

disease_dict = {'00' : '정상',
                
                # 병해 a
                'a1' : '딸기잿빛곰팡이병',
                'a2' : '딸기흰가루병',
                'a3' : '오이노균병',
                'a4' : '오이흰가루병',
                'a5' : '토마토흰가루병',
                'a6' : '토마토잿빛곰팡이병',
                'a7' : '고추탄저병',
                'a8' : '고추흰가루병',
                'a9' : '파프리카흰가루병',
                'a10' : '파프리카잘록병',
                'a11' : '시설포도탄저병',
                'a12' : '시설포도노균병',
                
                # 생리장애 b
                'b1' : '냉해피해',
                'b2' : '열과',
                'b3' : '칼슘결핍',
                'b4' : '일소피해',
                'b5' : '축과병',
                'b6' : '다량원소결핍 (N)',
                'b7' : '다량원소결핍 (P)',
                'b8' : '다량원소결핍 (K)',
                
                # 보호제 처리반응 c
                'c1' : '딸기잿빛곰팡이병반응',
                'c2' : '딸기흰가루병반응',
                'c3' : '오이노균병반응',
                'c4' : '오이흰가루병반응',
                'c5' : '토마토흰가루병반응',
                'c6' : '토마토잿빛곰팡이병반응',
                'c7' : '고추탄저병반응',
                'c8' : '고추흰가루병반응',
                'c9' : '파프리카흰가루병반응',
                'c10' : '파프리카잘록병반응',
                'c11' : '시설포도탄저병반응',
                'c12' : '시설포도노균병반응'
               }

            # 시설 과채류
grow_dict = {11:'유묘기',
             12:'생장기',
             13:'착화/과실기',
             
             # 시설 과수류 (포도)
             21:'발아기',
             22:'개화기',
             23:'신초생장기',
             24:'과실성숙기',
             25:'수확기',
             26:'휴면기'}

risk_dict = {0:'정상',
             1:'초기',
             2:'중기',
             3:'말기'}